# Metric Viewer
In this notebook we will view the metrics of all predictions

## Imports

In [ ]:
import evalseg
from evalseg.metrics import MME,MultiClassMetric
import numpy as np
import pandas as pd
from ipywidgets import interact, interact_manual, IntSlider
import os

## Variables

In [ ]:
root_data = '../datasets'

## Run Metric

In [ ]:
@interact
def select_dataset(dataset_name=sorted([d for d in os.listdir(root_data) if os.path.isdir(f'{root_data}/{d}')])):
    global dataset
    evalseg.reload()
    dataset = evalseg.io.NibDataset(f'{root_data}/{dataset_name}')

    @interact_manual
    def select_case(case=sorted(dataset.get_available_ids())):
        dataset.load_all_of_case(case)
        gto, gto_voxelsize = dataset.get_groundtruth(case)
        cto, cto_voxelsize = dataset.get_CT(case)
        gtmax = (gto.sum(axis=2).sum(axis=1).argmax(), gto.sum(axis=2).sum(axis=0).argmax(), gto.sum(axis=1).sum(axis=0).argmax())

        @interact
        def axselect(ax=['all','z', 'x', 'y']):
            axi = {'all': -1, 'x': 0, 'y': 1, 'z': 2}[ax]
            @interact
            def onlyMaxFrame(frame=IntSlider(-1,min=-1,max=-1) if ax=='all' else IntSlider(gtmax[axi], min=0, max=gto.shape[axi]-1)):
                global data, gt,ct, voxelsize
                gt, voxelsize = evalseg.geometry.slice(gto, gto_voxelsize, axi, [frame])
                ct, _ = evalseg.geometry.slice(cto, None, axi, [frame])
                data = {p: evalseg.geometry.slice(dataset.get_prediction(p, case)[0], None, axi, [frame])[0] for p in dataset.get_prediction_methods()}
                ctlbl = f"{dataset_name.split('_')[1]} {case}"
                if ax!='all':
                    ctlbl = f"{ctlbl} {ax}={frame}"
                    evalseg.ui.multi_plot_2d(ct, gt, data, spacing=voxelsize, ctlbl= ctlbl, z_titles= [frame])
                else:
                    return
                    # evalseg.ui3d.multi_plot_3d(ct, gt, data, spacing=voxelsize, ctlbl= ctlbl, z_titles= [frame])
                    pass #TODO
                mme = evalseg.metrics.MultiClassMetric(evalseg.metrics.MME,dataset.num_labels)
                mme.set_reference(gt, voxelsize)
                res = mme.evaluate_multi(data)
                evalseg.ui.plot_metric_multi(res, name=dataset_name)


In [ ]:
mme = evalseg.metrics.MultiClassMetric(evalseg.metrics.MME,dataset.num_labels)
mme.set_reference(gt, voxelsize)

In [ ]:
evalseg.reload()

multi_plot=evalseg.ui3d.plot_3d(ct, gt, 
        data['WCE'],
         spacing=voxelsize,zoom2segments=0,scaledown=(1,1,1),
         show_ct=False,
         show_gt=0,
         show_pred=1
         )

# evalseg.ui3d.multi_plot_3d(None, gt, {'CE':data['CE']}, spacing=voxelsize)

In [ ]:
ct.shape

In [ ]:
multi_plot.sync_camera_view()
# for i,plot in enumerate(multi_plot.plots):
    # if i>0:
    #     plot.camera_fov=30
multi_plot.remove_colorbar()
multi_plot.plots[0].camera_fov=30
multi_plot.plots[0].camera=b[0]

In [ ]:
multi_plot.plots[0]

In [ ]:
# multi_plot.plots[0].clipping_planes=[[1,0,0,-100],[-1,0,0,150],[0,1,0,-100],[0,0,1,-90]]
multi_plot.plots[2].clipping_planes=[[0, -1, 0, 50], [0, 0, -1, 95], [1, 0, 0, -50], [-1, 0, 0, 90]]

In [ ]:
b=[p.camera for p in multi_plot.plots]

In [ ]:
multi_plot.plots[2].camera=b[0]

In [ ]:
plt=multi_plot.plots[2]
plt.clipping_planes=[[1,0,0,-100],[-1,0,0,150],[0,1,0,-50],[0,0,1,-100]]

In [ ]:
import k3d
ct=cto.copy()
ct_bounds = [0, ct.shape[0], 0, ct.shape[1], 0, ct.shape[2]]
ct[ct>400]=0
ct[gt>0]=500
plot_ct = k3d.plot(grid=ct_bounds,  grid_auto_fit=False, camera_auto_fit=0)
plot_ct.display()
plot_ct += k3d.volume(ct.astype(np.float32),
        #                       alpha_coef=100,
                              color_map=k3d.colormaps.matplotlib_color_maps.Turbo,
        #                       #   color_map=k3d.colormaps.paraview_color_maps.Nic_Edge,
        #                       #   color_map=k3d.colormaps.matplotlib_color_maps.gist_rainbow,
                              scaling=voxelsize,
                              color_range=[100, 400],
                              bounds=ct_bounds,
        #                       #   gradient_step=1,
                              name='ct')

In [ ]:
import k3d
evalseg.reload()
plot = k3d.plot()

cube_1 = evalseg.ui3d.k3d_tools.Cube([10, 20, 30], size=[10,20,30]).mesh
cube_1.wireframe=True
plot += cube_1

plot.display()
